In [180]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
pd.set_option('display.max_columns', None)

In [181]:
data = pd.read_csv('learningSet.csv')
data.shape

/var/folders/dm/hhj01rls7tz3g5v4m3hhj1bw0000gn/T/ipykernel_7884/3302850674.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('learningSet.csv')


(95412, 481)

In [202]:
data['MAILCODE'] = data['MAILCODE'].apply(lambda x: x.replace(" ", "A"))

In [203]:
data = data.apply(lambda x: x.replace(" ", np.NaN))

# Complete the following steps on the categorical columns in the dataset:

- Check for null values in all the columns

In [185]:
data.isnull().sum()

ODATEDW       0
OSOURCE     928
TCODE         0
STATE         0
ZIP           0
           ... 
MDMAUD_R      0
MDMAUD_F      0
MDMAUD_A      0
CLUSTER2    132
GEOCODE2    319
Length: 481, dtype: int64

In [186]:
categorical = data.select_dtypes(object)
categorical.head()
categorical.shape

(95412, 74)

In [187]:
nulls_percent_df = pd.DataFrame(categorical.isna().sum()/len(categorical)).reset_index()
nulls_percent_df.columns = ['column_name', 'nulls_percentage']
columns_w_nulls = nulls_percent_df[nulls_percent_df['nulls_percentage']!=0].reset_index(drop= True)
columns_w_nulls

,column_name,nulls_percentage
0,OSOURCE,0.009726
1,PVASTATE,0.984719
2,NOEXCH,0.000073
3,RECINHSE,0.929747
4,RECP3,0.978860
...,...,...
59,RFA_21,0.369052
60,RFA_22,0.268813
61,RFA_23,0.589800
62,RFA_24,0.387509


- Exclude the following variables by looking at the definitions. Create a new empty list called `drop_list`. We will append this list and then drop all the columns in this list later:
    - `OSOURCE` - symbol definitions not provided, too many categories
    - `ZIP` - we are including state already

In [188]:
drop_list = []
drop_list.extend(["OSOURCE", "ZIP"])
drop_list

['OSOURCE', 'ZIP']

- Identify columns that have over 50% missing values.

In [189]:
columns_above_threshold = columns_w_nulls[columns_w_nulls['nulls_percentage']>0.5]
columns_above_threshold['column_name']
drop_columns_list = list(columns_above_threshold['column_name'])
drop_list.extend(drop_columns_list)
len(drop_list)

38

KEEPING VETERANS AND SOLIH, TO MATCH THE NEEDS

In [190]:
names_to_remove = ["VETERANS", "SOLIH"]

for name in names_to_remove:
    if name in drop_list:
        drop_list.remove(name)
    
len(drop_columns_list)

36

- Remove those columns from the dataframe

In [191]:
data = data.drop(drop_list, axis=1)


In [192]:
data.shape

(95412, 445)

## - Perform all of the cleaning processes from the Lesson.

 - Remove columns with null values greater than a specified threshold (0.25 of all the dataframe) KEEPING: "WEALTH1", "WEALTH2" ,"VETERANS", "SOLIH"

In [193]:
nulls_percent_df = pd.DataFrame(data.isna().sum()/len(data)).reset_index()
nulls_percent_df.columns = ['column_name', 'nulls_percentage']

In [194]:
columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>0.25]
columns_above_threshold['column_name']
drop_columns_list = list(columns_above_threshold['column_name'])
len(drop_columns_list)

83

In [195]:
names_to_remove = ["WEALTH1", "WEALTH2" ,"VETERANS", "SOLIH"]

for name in names_to_remove:
    if name in drop_columns_list:
        drop_columns_list.remove(name)
    
len(drop_columns_list)

79

In [196]:
data = data.drop(drop_columns_list, axis=1)
data.shape

(95412, 366)

REPLACE NAN IN VETERANS FOR N

In [197]:
data["VETERANS"].unique()

array([nan, 'Y'], dtype=object)

In [198]:
data['VETERANS'] = data['VETERANS'].fillna("N")
data["VETERANS"].unique()

array(['N', 'Y'], dtype=object)

REDUCING STATES

In [199]:
df = pd.DataFrame(data['STATE'].value_counts()).reset_index()
df.columns = ['state', 'count']
other_states = list(df[df['count']<2500]['state'])
def clean_state(x):
    if x in other_states:
        return 'other'
    else:
        return x
    
data['STATE'] = list(map(clean_state, data['STATE']))

In [200]:
data["STATE"].value_counts()

other    30457
CA       17343
FL        8376
TX        7535
IL        6420
MI        5654
NC        4160
WA        3577
GA        3403
IN        2980
WI        2795
MO        2712
Name: STATE, dtype: int64

- Reduce the number of categories in the column `GENDER`. The column should only have either "M" for males, "F" for females, and "other" for all the rest
    - Note that there are a few null values in the column. We will first replace those null values using the code below:

    ```python
    print(categorical['GENDER'].value_counts())
    categorical['GENDER'] = categorical['GENDER'].fillna('F')
    ```

In [205]:
data['GENDER'].isna().sum()

2957

In [206]:
data['GENDER'] = data['GENDER'].fillna("F")
data["GENDER"].unique()

array(['F', 'M', 'C', 'U', 'J', 'A'], dtype=object)

In [207]:
data['GENDER'].value_counts()

F    54234
M    39094
U     1715
J      365
C        2
A        2
Name: GENDER, dtype: int64

In [209]:
df = pd.DataFrame(data['GENDER'].value_counts()).reset_index()
df.columns = ['gender', 'count']
df

,gender,count
0,F,54234
1,M,39094
2,U,1715
3,J,365
4,C,2
5,A,2


In [219]:
unique_values = df['gender'].unique()
other_genders = [value for value in unique_values if value not in ["F", "M"]]
other_genders


['U', 'J', 'C', 'A']

In [220]:
def clean_gender(x):
    if x in other_genders:
        return 'other'
    else:
        return x
    
data['GENDER'] = list(map(clean_gender, data['GENDER']))
data["GENDER"].value_counts()

F        54234
M        39094
other     2084
Name: GENDER, dtype: int64